# Lathe of Heaven

## Introduction

This workbook is intended to develop and test the conceptual framework of terrain generation.


## Set Variables

In MightyBob's separate-sampling experiment, rough/strength pairs of (1.6, 0.4) (5, 0.2) and (24, 0.02) were good for 3 octaves. The final 3 results were added and then multiplied by 0.4.

30,000 × 15,000 pixels is probably the lowest pixel size we can use for a roughly 1px to 1km resolution. 36,000 × 18,000 is evenly divisible by 360 and multiples thereof, but is approximaetly 130% larger than necessary. 40,000 × 20,000 is closest to the actual circumferential dimensions of the earth but because Earth is not a perfect sphere, but this yields approximately 300 million more pixels than required to model the surface area (~160% larger than necessary).


In [2]:
# Import libraries.

import numpy as np
import opensimplex as osi

# Define variables.

RADIUS: int = 6378100  # Radius of the world in meters. The approximate radius of Earth is 6378100 m.
FEATURE_SIZE: float = (
    RADIUS * 0.5
)  # Determines the "coherence" of the random noise; affects the size of discrete landmasses. A good size for Earth-like continents is 0.5× the radius.
ZMIN: int = round(
    number=-(RADIUS * 0.0015)
)  # The lowest elevation in the world, in meters. The deepest oceanic trench on Earth is approximately -10,300 m.
ZMAX: int = round(
    number=RADIUS * 0.0015
)  # The highest elevation in the world, in meters. The highest peak on Earth is approximately 8700 m.
ZRANGE: float = ZMAX - ZMIN
ZTILT: float = 23.4  # Used in mapping spherical coordinates to lat-lon coordinates in a Coordinate Reference System (CRS). The present z-axis tilt of the Earth is approximately 23.4 degrees.
OCEAN_PERCENT: float = 0.55  # Sets the point of elevation 0 as a relative percent of the range from zmin to zmax during rescaling.
OCEAN_POINT: float = OCEAN_PERCENT * ZRANGE
RECURSION: int = 9  # The number of recursions used in creating the icosphere mesh. 9 yields 2,621,442 points and 5,242,880 cells. Surface area of the Earth is approximately 514 million km2.
OCTAVES: int = 8  # Sets the number of iterations to use for noise sampling, resulting in a more complex output.
INIT_ROUGHNESS: float = (
    1.5  # Sets the initial roughness (frequency) of the first octave of noise.
)
INIT_STRENGTH: float = (
    0.4  # Sets the initial strength (amplitude) of the first octave of noise.
)
ROUGHNESS: float = 2.5  # Multiplies roughness by this much per octave.
STRENGTH: float = 0.5  # Multiplies strength by this much per octave; aka. persistence.
PLATESNUM: int = round(
    number=RADIUS / 1_000_000
)  # The number of tectonic plates to generate. The Earth has approximately 12 major plates.

# Set world seed.

seed_rng: np.random = np.random.default_rng()
world_seed: int = seed_rng.integers(low=0, high=999999)
osi.seed(seed=world_seed)

## Initialize Meshes and Arrays


In [3]:
# Import libraries.

import pyvista as pv

# import pyvistaqt as pvqt

# Initialize world mesh.

world_mesh = pv.Icosphere(radius=RADIUS, center=(0.0, 0.0, 0.0), nsub=RECURSION)

# Perform axial rotation.

world_mesh.points = pv.axis_rotation(
    world_mesh.points, ZTILT, inplace=False, deg=True, axis="z"
)

print(world_mesh, "\r\n")
print(f"Recursion Level: {RECURSION}\r")
print(f"Shape of Points Array: {np.shape(world_mesh.points)}\r")
print(f"Shape of Faces Array: {np.shape(world_mesh.faces)}\r")

PolyData (0x2469dec0a60)
  N Cells:    5242880
  N Points:   2621442
  N Strips:   0
  X Bounds:   -6.378e+06, 6.378e+06
  Y Bounds:   -6.378e+06, 6.378e+06
  Z Bounds:   -6.378e+06, 6.378e+06
  N Arrays:   0 

Recursion Level: 9
Shape of Points Array: (2621442, 3)
Shape of Faces Array: (20971520,)


## Generate Elevation Noise

_Note from MightyBob: Adding +1 to elevation moves negative values in the 0-1 range. Multiplying by 0.5 drags any values > 1 back into the 0-1 range. I'm not sure if multiplying by the radius is the proper thing to do in my next implementation._


In [4]:
# Import libraries.

from numba import prange


def sample_noise(
    points, roughness, strength, feature_size, radius
) -> np.ndarray[np.float64]:
    elevations: np.ndarray[np.float64] = np.ones(shape=len(points), dtype=np.float64)
    rough_verts = points * roughness

    for v in prange(len(rough_verts)):
        elevations[v] = (
            osi.noise4(
                x=rough_verts[v][0],
                y=rough_verts[v][1],
                z=rough_verts[v][2],
                w=feature_size,
            )
            / feature_size
        )

    return (elevations + 1) * 0.5 * strength * radius


def sample_octaves(
    points,
    octaves,
    init_roughness,
    init_strength,
    roughness,
    strength,
    feature_size,
    radius,
) -> np.ndarray[np.float64]:
    # Initialize elevations array.
    elevations: np.ndarray[np.float64] = np.zeros(shape=len(points), dtype=np.float64)

    for i in prange(octaves):
        elevations += sample_noise(
            points=points,
            roughness=init_roughness / radius,
            strength=init_strength / radius,
            feature_size=feature_size,
            radius=radius,
        )
        init_roughness *= roughness
        init_strength *= strength

    return elevations


raw_elevations: np.ndarray[np.float64] = sample_octaves(
    points=world_mesh.points,
    octaves=OCTAVES,
    init_roughness=INIT_ROUGHNESS,
    init_strength=INIT_STRENGTH,
    roughness=ROUGHNESS,
    strength=STRENGTH,
    feature_size=FEATURE_SIZE,
    radius=RADIUS,
)


# Display raw elevations.

print("Raw Elevations:\r\n")
print(raw_elevations, "\r\n")
print(f"Shape: {np.shape(raw_elevations)}\r")
print(f"Minima: {np.min(raw_elevations)}\r")
print(f"Maxima: {np.max(raw_elevations)}\r")

Raw Elevations:

[0.39843751 0.39843751 0.39843747 ... 0.39843749 0.39843749 0.39843749] 

Shape: (2621442,)
Minima: 0.3984374435215887
Maxima: 0.3984375697637237


## Apply Elevation Scalars to Mesh


In [78]:
# Rescale elevations.

e_min: np.float64 = np.min(raw_elevations)
e_max: np.float64 = np.max(raw_elevations)
e_range: np.float64 = e_max - e_min

elevations = ((raw_elevations - e_min) / e_range) * ZRANGE + ZMIN

# Display scaled elevations.

print("Scaled Elevations:\r\n")
print(f"Shape: {np.shape(elevations)}\r")
print(f"Minima: {np.min(elevations)}\r")
print(f"Maxima: {np.max(elevations)}\r\n")

# Generate elevation scalars.

elevation_scalars: np.ndarray[np.float64] = (elevations + RADIUS) / RADIUS

# Display elevation scalars.

print("Elevation Scalars:\r\n")
print(f"Shape: {np.shape(elevation_scalars)}\r")
print(f"Minima: {np.min(elevation_scalars)}\r")
print(f"Maxima: {np.max(elevation_scalars)}\r\n")

# Apply elevation scalars to world mesh.

world_mesh.points[:, 0] *= elevation_scalars
world_mesh.points[:, 1] *= elevation_scalars
world_mesh.points[:, 2] *= elevation_scalars

# Test elevation values.

elevation_values = world_mesh.points
elevation_values = 

# Display test elevation values.

print("Elevation Test Values:\r\n")
print(f"Shape: {np.shape(elevation_values)}\r")
print(f"Minima: {np.min(elevation_values)}\r")
print(f"Maxima: {np.max(elevation_values)}\r\n")

# Set elevations dataset.

world_mesh.point_data["Elevation"] = elevations

# Display elevation dataset.

print("Elevation Dataset:\r\n")
print(f"Shape: {np.shape(world_mesh.point_data["Elevation"])}\r")
print(f"Minima: {np.min(world_mesh.point_data["Elevation"])}\r")
print(f"Maxima: {np.max(world_mesh.point_data["Elevation"])}\r\n")

Scaled Elevations:

Shape: (2621442,)
Minima: -9567.0
Maxima: 9567.0

Elevation Scalars:

Shape: (2621442,)
Minima: 0.9985000235179756
Maxima: 1.0014999764820245

Elevation Test Values:

Shape: (2621442, 3)
Minima: -6411161.623752146
Maxima: 6428978.781782424

Elevation Dataset:

Shape: (2621442,)
Minima: -9567.0
Maxima: 9567.0



## Display Raw Elevations


In [76]:
# Import libraries.

import cmocean.cm as cmo

# Set global pyvista options.

pv.plotter._ALL_PLOTTERS.clear()
pv.set_plot_theme("dark")
pv.global_theme.anti_aliasing = "ssaa"
pv.global_theme.cmap = "topo"
pv.global_theme.lighting = True

# Set plotter attributes.

pl = pv.Plotter(
    off_screen=True,
    notebook=True,
    shape=(1, 1),
    border=True,
    border_color="black",
    window_size=(1024, 768),
    line_smoothing=True,
    polygon_smoothing=True,
    lighting="light kit",
    image_scale=1,
)

pl.clear()
pl.enable_anti_aliasing()
pl.enable_hidden_line_removal(all_renderers=True)

scalar_args: dict = dict(
    interactive=False,
    height=0.75,
    vertical=True,
    position_x=0.75,
    position_y=0.15,
    title_font_size=20,
    label_font_size=14,
    shadow=False,
    n_labels=9,
    italic=False,
    fmt="%.1f",
    font_family="times",
)

annotations = {}


# Add actors.

world_mesh.compute_normals(cell_normals=True, point_normals=True, inplace=True)
globe_mesh = world_mesh.warp_by_scalar(factor=-50)

pl.add_mesh(
    globe_mesh,
    name="Base Terrain",
    scalars="Elevation",
    scalar_bar_args=scalar_args,
    show_scalar_bar=True,
    annotations=annotations,
    style="surface",
    smooth_shading=True,
    show_edges=False,
    edge_color="red",
    line_width=1,
    cmap="cmo.topo",
    lighting=True,
    pickable=True,
    preference="cell",
)

# pl.add_title(title="Base Terrain", font_size=18, color=None, font="times", shadow=True)

# Render scene.

pl.render()
pl.show()

Widget(value='<iframe src="http://localhost:64386/index.html?ui=P_0x246814584a0_0&reconnect=auto" class="pyvis…

## Define Tectonic Plates


In [79]:
import heapq
from scipy.spatial import KDTree


def a_star(world_mesh, N):
    # Get the N highest and lowest points
    highest_points = world_mesh.points[
        np.argsort(world_mesh.point_data["Elevation"], kind="heapsort")[-N:]
    ]
    lowest_points = world_mesh.points[
        np.argsort(world_mesh.point_data["Elevation"], kind="heapsort")[:N]
    ]

    print(f"Highest Points ({len(highest_points)}):\r\n{highest_points}\r\n")
    print(f"Lowest Points ({len(lowest_points)}):\r\n{lowest_points}\r\n")

    # Create a KDTree for efficient nearest neighbor search
    kd_tree = KDTree(world_mesh.points)

    # Initialize the open and closed sets
    open_set = []
    closed_set = set()

    # Initialize the start and goal nodes
    start_node = highest_points[0]
    goal_node = lowest_points[0]

    # Initialize the g and f scores
    g_scores = {start_node: 0}
    f_scores = {start_node: np.linalg.norm(start_node - goal_node)}

    # Add the start node to the open set
    heapq.heappush(open_set, (f_scores[start_node], start_node))

    # Initialize the path dictionary
    path = {}

    while open_set:
        # Get the node with the lowest f score
        current_node = heapq.heappop(open_set)[1]

        # Check if the current node is the goal node
        if current_node == goal_node:
            # Reconstruct the path
            path_points = [current_node]
            while current_node in path:
                current_node = path[current_node]
                path_points.append(current_node)
            return np.array(path_points[::-1])

        # Add the current node to the closed set
        closed_set.add(current_node)

        # Find the neighbors of the current node
        _, neighbor_indices = kd_tree.query(current_node, k=8)

        for neighbor_index in neighbor_indices:
            neighbor_node = world_mesh.points[neighbor_index]

            # Skip if the neighbor node is in the closed set
            if neighbor_node in closed_set:
                continue

            # Calculate the tentative g score
            tentative_g_score = g_scores[current_node] + np.linalg.norm(
                current_node - neighbor_node
            )

            # Check if the neighbor node is not in the open set or the tentative g score is lower
            if (
                neighbor_node not in g_scores
                or tentative_g_score < g_scores[neighbor_node]
            ):
                # Update the path and g score
                path[neighbor_node] = current_node
                g_scores[neighbor_node] = tentative_g_score
                f_scores[neighbor_node] = tentative_g_score + np.linalg.norm(
                    neighbor_node - goal_node
                )

                # Add the neighbor node to the open set
                heapq.heappush(open_set, (f_scores[neighbor_node], neighbor_node))

    # Return an empty path if no path is found
    return np.array([])


# Usage example
N = 10
path = a_star(world_mesh, N)
print(path)

Highest Points (10):
[[ 4215690.29981996  2334249.18958404 -4620367.83970102]
 [ 4274899.19719654  2448626.76994013 -4489124.15855907]
 [ 4096132.73644269  2345397.82840017 -4736164.80746869]
 [ 4304304.38032016  2374524.8416193  -4502890.07401394]
 [ 4297060.86079323  2356402.96692671 -4522029.83032679]
 [ 4173144.70273219  2435849.21387744 -4606422.15241285]
 [ 4159640.59219022  2462036.28725928 -4604586.29784729]
 [ 4300882.16015547  2365580.8404145  -4513538.17910941]
 [ 4177161.51572483  2445046.49066601 -4597849.32297825]
 [ 4156215.22251771  2453210.82339648 -4616884.96233021]]

Lowest Points (10):
[[-4690798.59575405 -2832037.84806874  2873743.28695359]
 [-4686932.05241551 -2825853.01371102  2884738.18402413]
 [-4699987.85151885 -2824003.41327202  2868130.7648927 ]
 [-4695017.50180379 -2821016.96202919  2878166.80226208]
 [-4704333.78390568 -2813067.90718309  2873043.50158866]
 [-3383361.39313086 -4481146.13977564  2637896.83342824]
 [-3400410.43525503 -4459337.17025377  265110

TypeError: unhashable type: 'pyvista_ndarray'